In [ ]:
# Generate MAPF maps with designated size and obstacle density,
# create random start and goal locations and find paths with EECBS (https://github.com/Jiaoyang-Li/EECBS/tree/main)

In [5]:
import subprocess
from pathlib import Path
import numpy as np
from pathlib import Path
import sys
sys.path.append('../')
import config

In [6]:
def flood_fill(matrix, x, y, old_value, new_value):
    """Performs the flood fill algorithm"""
    if x < 0 or x >= matrix.shape[0] or y < 0 or y >= matrix.shape[1]:
        return
    if matrix[x, y] != old_value:
        return
    matrix[x, y] = new_value
    flood_fill(matrix, x+1, y, old_value, new_value)
    flood_fill(matrix, x-1, y, old_value, new_value)
    flood_fill(matrix, x, y+1, old_value, new_value)
    flood_fill(matrix, x, y-1, old_value, new_value)

def generate_map(width, height, density, tolerance=0.005):
    """Generates a grid map with specified width, height, and obstacle density"""
    
    iteration = 0
    max_iterations = config.MAX_ITERATIONS
    
    while iteration < max_iterations:
        matrix = np.random.choice([0, 1], size=(width, height), p=[1-density, density])
        
        # Clone the matrix to keep the original for calculation purposes
        filled_matrix = matrix.copy()
        
        # Use flood fill from top-left to mark all reachable cells with value 2
        flood_fill(filled_matrix, 0, 0, 0, 2)
        
        # Calculate the reachable free space
        total_free_space = np.sum(filled_matrix == 2)
        total_space = width * height
        actual_density = 1 - total_free_space/total_space
        
        # If the actual density is close to desired density, finalize the matrix
        if abs(actual_density - density) < tolerance:
            # After flood fill, change all 0 (unreachable free cells) to 1 (obstacles)
            filled_matrix[filled_matrix == 0] = 1
            
            # Change the 2's back to 0's
            filled_matrix[filled_matrix == 2] = 0
            
            return filled_matrix
        
        iteration += 1
    
    # If we couldn't achieve the desired density in max_iterations
    raise ValueError(f"Unable to generate a grid with the desired density of {density} after {max_iterations} iterations.")

def save_env_map_to_file(map, file_path):
    with open(file_path, 'w') as file:
        file.write("type octile\n")
        file.write(f"height {map.shape[0]}\n")
        file.write(f"width {map.shape[1]}\n")
        file.write("map\n")
        for row in map:
            line = ''.join(['.' if cell == 0 else '@' for cell in row]) + '\n'
            file.write(line)


def generate_instance(dim=40, density=0.3, num_instances=1):
    folder = Path(f'./temp/dim{dim}_density{density}') # base directory:'./temp/'
    folder.mkdir(parents=True, exist_ok=True) 
    for i in range(num_instances):
        generator = generate_map(dim, dim, density)
        env_map_file_path = folder / f'random_{dim}_{density}_case_{i+1}.map'
        save_env_map_to_file(generator, env_map_file_path)



In [ ]:
def run_eecbs_on_files(base_dir, k=64, t=60, suboptimality=1.2):
 
    # Assuming the base_dir is 'instances' or any other directory containing your generated folders
    folders = [folder for folder in Path(base_dir).iterdir() if folder.is_dir()]

    for folder in folders:
        
        map_files = list(folder.glob('*.map'))

        for map_file in map_files:
            output_base = f'{map_file.stem}'
            cmd = [
                config.PATH_TO_EECBS,   # Path to the EECBS executable
                '-m', str(map_file),
                '-a', f'{folder}/{output_base}_agents_{k}.scen',
                '-o', f'{folder}/{output_base}_agents_{k}_stats.csv',           
                '--outputPaths', f'{folder}/{output_base}_agents_{k}_paths.txt', 
                '-k', str(k),
                '-t', str(t),
                '--suboptimality', str(suboptimality)
            ]
            subprocess.run(cmd)




In [ ]:
generate_instance(dim=40, density=0.3, num_instances=100)

In [ ]:
base_directory = './temp/'
for i in [4,8,16,32,64]:
    run_eecbs_on_files(base_directory,i)